In [22]:
import numpy as np
import pandas as pd
import random
from joblib import Parallel, delayed

In [101]:
# 生成随机矩阵X_n
n = 1000    # 样本量
p = 1000    # 变量数

# 创建协方差矩阵
cov_matrix = np.full((p, p), 0.5)  # 创建一个所有元素都是 0.5 的矩阵
np.fill_diagonal(cov_matrix, 2.0)  # 将对角线元素设置为 2.0

mean_vector                = np.zeros(p)  # 均值向量
X                          = np.random.multivariate_normal(mean_vector, cov_matrix, n)
# 奇异值分解
U, S, Vt = np.linalg.svd(X)

# 计算条件数
condition_number = S[0] / S[-1]
print("条件数:", condition_number)

# 打印奇异值
# S_adjusted = np.copy(S)
# for i in range(len(S)):
#         S_adjusted[i] /= 10**(0.00225*i)   # 或者选择其他缩放因子
# print("最大奇异值:", np.max(S_adjusted))
# print("最小奇异值:", np.min(S_adjusted))
# print("条件数:", np.max(S_adjusted)/np.min(S_adjusted))
# Sigma = np.zeros((U.shape[0], Vt.shape[0]))
# np.fill_diagonal(Sigma, S_adjusted)
# X =  U @ Sigma @ Vt

条件数: 98.46354442674601


In [84]:
# 定义β向量
beta = np.zeros(p)  # 初始化β向量为0

# 根据条件设置β向量的值
beta[0:5]   = 2.0   # βi = 2.0, i = 1, 2, 3, 4, 5
beta[5:10]  = -2.0  # βi = -2.0, i = 6, 7, 8, 9, 10
beta[10:15] = 1.0   # βi = 1.0, i = 11, 12, 13, 14, 15
beta[15:20] = -1.0  # βi = -1.0, i = 16, 17, 18, 19, 20
# beta[20:25] = 5.0   # βi = 5.0, i = 21, 22, 23, 24, 25

# Generate 20 random indices
indices = np.random.randint(low=0, high=p, size=20)

# Get the indices of non-zero elements in beta
nonzero_indices = np.nonzero(beta)[0]

# print(nonzero_indices)

# 初始化random_beta为零向量
random_beta = np.zeros(p)

# Assign the non-zero elements randomly
# 此处假设的意图是将beta中的非零元素随机分配到random_beta中
# 但这样的操作逻辑上存在问题，因为indices的长度和nonzero_indices可能不同
# 为了简化，这里我们只是简单地复制beta到random_beta
random_beta[indices] = beta[nonzero_indices]

# Print the randomly assigned beta
# print(random_beta)
beta = random_beta
nonzero_indices = np.nonzero(beta)[0]
print(nonzero_indices)

[   4   78  192  291  301  306  454  534  635  640  709  776  817  823
  846  948 1119 1147 1250 1341]


In [49]:
theta = Vt.T @ Vt @ beta

In [102]:
# 生成均值为0方差为4的n维向量en
mean    = 0  # 均值
std_dev = 2  # 标准差，方差为标准差的平方，即4

# 生成向量
e = np.random.normal(mean, std_dev, n)
norm_e = np.linalg.norm(e)
Y = np.dot(X, beta) + e
print(norm_e)

64.75012571858161


In [26]:
tau   = 1
delta = tau*norm_e
s1, s2 = X.shape

In [27]:
# 最小二乘法
def least_square(X, Y):
    beta = np.linalg.lstsq(X, Y, rcond=None)[0]
    return beta
beta_LS = least_square(X, Y)
norm_LS = np.linalg.norm(beta_LS - beta)
print(norm_LS)
sigma_LS = np.linalg.norm(Y - np.dot(X, beta_LS))
sigma_LS = sigma_LS / np.sqrt(s2)
print(sigma_LS)

4.797995245748872
2.4734865018731572e-14


In [103]:
# Ridge Regression
C = 200000
m = 100
def compute_mse(c, X, Y):
    alpha = c / s1
    I = np.eye(s2)
    beta_r = np.linalg.inv(X.T @ X + alpha * I) @ X.T @ Y
    mse = np.linalg.norm(beta_r - beta)
    return mse

# 假设 X, Y, C 已经定义
mse_list = Parallel(n_jobs=-1)(delayed(compute_mse)(c, X, Y) for c in range(0, C + 1, m))

best_c = np.argmin(mse_list)*m
print("The value of C that minimizes the mean squared error is:", best_c)

The value of C that minimizes the mean squared error is: 131100


In [104]:
# 岭回归
def ridge(X, Y, C):
    alpha  = C/s1
    I      = np.eye(s2)
    gen    = np.linalg.inv(X.T @ X + alpha*I) @ X.T
    beta   = gen @ Y
    beta_debias = (I + alpha*np.linalg.inv(X.T @ X + alpha*I))@beta
    gen_debias  = (I + alpha*np.linalg.inv(X.T @ X + alpha*I))@gen
    return beta, beta_debias, gen, gen_debias

beta_Ridge, beta_Ridge_debias, gen_Ridge, gen_Ridge_debias = ridge(X, Y, best_c)
norm_Ridge = np.linalg.norm(beta_Ridge - beta)
norm_Ridge_debias = np.linalg.norm(beta_Ridge_debias - beta)
print(norm_Ridge, norm_Ridge_debias)
sigma_Ridge = np.linalg.norm(Y - np.dot(X, beta_Ridge))
sigma_Ridge = sigma_Ridge / np.sqrt(s2)
print(sigma_Ridge)
sigma_Ridge_debias = np.linalg.norm(Y - np.dot(X, beta_Ridge_debias))
sigma_Ridge_debias = sigma_Ridge_debias / np.sqrt(s2)
print(sigma_Ridge_debias)

4.593042635195277 4.6573841549331805
0.5824834669608071
0.1621471153397847


In [34]:
# Landweber回归
t        = 5e-7    # 步长
max_iter = 10000   # 最大迭代次数

def landweber(X, Y, t, max_iter):
    s = X.shape
    m = s[1]
    beta_cur = np.zeros(m)
    for k in range(1, max_iter):
        beta_prev = beta_cur
        beta_cur = beta_prev + t * np.dot(X.T, (Y - np.dot(X, beta_prev)))
        if np.linalg.norm(Y - np.dot(X, beta_cur)) <= delta:
            break
    return beta_cur, k
def landweber_debias(X, Y, t, max_iter):
    s = X.shape
    m = s[1]
    beta_cur = np.zeros(m)
    for k in range(1, max_iter):
        beta_prev = beta_cur
        beta_cur = beta_prev + t * np.dot(X.T, (Y - np.dot(X, beta_prev)))
        if k == max_iter:
            break
    return beta_cur
beta_Landweber, k_Landweber = landweber(X, Y, t, max_iter)
beta_Landweber_debias = landweber_debias(X, Y, t, 2*k_Landweber)
norm_Landweber    = np.linalg.norm(beta_Landweber - beta)
norm_Landweber_debias  = np.linalg.norm(beta_Landweber_debias - beta)  

print(norm_Landweber, k_Landweber, norm_Landweber_debias)
sigma_Landeweber = np.linalg.norm(Y - np.dot(X, beta_Landweber))
sigma_Landeweber = sigma_Landeweber / np.sqrt(s2)
print(sigma_Landeweber)
sigma_Landeweber_debias = np.linalg.norm(Y - np.dot(X, beta_Landweber_debias))
sigma_Landeweber_debias = sigma_Landeweber_debias / np.sqrt(s2)
print(sigma_Landeweber_debias)

4.9630165839947535 1740 4.7394313726118895
1.5882680978563277
0.9657833862100805
